Multi-GPU PageRank Demo
=======================

This example runs PageRank on HiBench data using RAPIDS. Before running this notebook, you need to start a DASK scheduler and DASK workers in command line.

1) dask-scheduler --scheduler-file [scheduler_file_path]

* [scheduler_file_path]: path to write scheduler access information in the json format (e.g. /home/USERID/cluster.json).

2) mpirun -np [number_of_workers] --machinefile [machine_address_file_path] dask-mpi --no-nanny --nthreads [number_of_threads] --local-directory [local_directory_path] --no-scheduler --scheduler-file [scheduler_file_path]

* [numbrer_of_workers]: number of DASK workers to start, set this to the number of GPUs on your machine.
* [machine_address_file_path]: path to the MPI machine address file, --machinefile [machine_address_file_path] can be skipped for single node systems.
* [numbrer_of_threads]: number of threads per worker, should be >= 2.
* [local_directory_path]: path to place temporary worker files.
* [scheduler_file_path]: path to read scheduler access information written by the DASK scheduler.

1) Import Files
=======================


In [ ]:
import os
import time

from dask.distributed import Client

import dask_cudf
import dask_cugraph as dcg

2) Set the Number of GPU Devices and File Paths
=======================

In [ ]:
number_of_devices = 2
scheduler_file_path = r"/home/USERID/cluster.json"
input_data_path = r"/datasets/pagerank/Input-bigdata/edges"

3) Define Utility Functions
=======================
set_visible maps a dask-mpi process to a single GPU device.

In [3]:
def set_visible(i, n):
    all_devices = list(range(n))
    visible_devices = ",".join(map(str, all_devices[i:] + all_devices[:i]))
    os.environ["CUDA_VISIBLE_DEVICES"] = visible_devices

4) Create a Client
=======================
Connect to the DASK scheduler.

In [4]:
start_time = time.time()  # start timing from here

client = Client(scheduler_file=scheduler_file_path,
                direct_to_workers=True)

5) Map One Worker to One GPU
=======================

In [5]:
devices = list(range(number_of_devices))
device_workers = list(client.has_what().keys())
assert len(devices) == len(device_workers)

[client.submit(set_visible, device, len(devices), workers=[worker])
    for device, worker in zip(devices, device_workers)]

[<Future: status: pending, key: set_visible-4f3af2f890d6a36d7e8217c80e27002c>,
 <Future: status: pending, key: set_visible-734ca7f9b333fe2c93c28d89ff836b50>]

6) Read Input Data
=======================
Need to replace /datasets/pagerank_demo/Input-bigdata/edges to the proper directory path

In [6]:
dgdf = dask_cudf.read_csv(input_data_path + r"/part-*",
                          delimiter='\t', names=['src', 'dst'],
                          dtype=['int32', 'int32'])

7) Run PageRank
=======================

In [ ]:
pagerank = dcg.mg_pagerank(dgdf)
print(pagerank)

5000


8) Close the Client and Report Execution Time
=======================

In [ ]:
client.close()

end_time = time.time()
print((end_time - start_time), "seconds")